# PROYECTO FINAL: NullStorm

## 1. MODELO OCR

In [1]:
from google.colab import files
from google.cloud import bigquery
import cv2
import numpy as np
import easyocr
import matplotlib.pyplot as plt
import re
import random
from datetime import datetime, timedelta

## 1.1. Insertar la imagen de entrada

In [2]:
def insert_img():

    print("Por favor, selecciona una imagen del vehículo:")
    uploaded = files.upload()

    if not uploaded:
        print("No se seleccionó ninguna imagen.")
        return

    # Leer la imagen subida
    image_path = next(iter(uploaded))
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

## 1.2. Detección de la placa

In [3]:
def detectar_placa(img):
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Aplicar un filtro bilateral para reducir el ruido mientras se preservan los bordes
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)
    # Detectar bordes utilizando el algoritmo Canny
    edged = cv2.Canny(bfilter, 30, 200)

    # Encontrar contornos en la imagen
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = keypoints[0] if len(keypoints) == 2 else keypoints[1]
    # Ordenar los contornos por área, de mayor a menor, y tomar los 10 más grandes
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    location = None
    for contour in contours:
        # Aproximar el contorno a un polígono
        approx = cv2.approxPolyDP(contour, 10, True)
        # Si el polígono tiene 4 vértices, asumimos que es la placa
        if len(approx) == 4:
            location = approx
            break

    if location is not None:
        # Crear una máscara para aislar la región de la placa
        mask = np.zeros(gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [location], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)

        # Extraer la región de la placa
        (x, y) = np.where(mask == 255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        placa_img = gray[x1:x2+1, y1:y2+1]

        return placa_img

    return None

## 1.3. Reconocer la matrícula y limpiar el resultado

In [4]:
def limpiar_placa(texto_placa):
    placa_limpia = re.sub(r'[^A-Z0-9]', '', texto_placa.upper())
    return placa_limpia

In [5]:
def reconocer_placa(placa_img):
    # Inicializar el lector OCR para español
    reader = easyocr.Reader(['es'])
    # Realizar OCR en la imagen de la placa
    result = reader.readtext(placa_img)
    if result:
        texto_crudo = result[0][-2]
        # Devolver el texto limpio
        texto_limpio = limpiar_placa(texto_crudo)
        return texto_limpio
    return None

In [12]:
def detectar_placa_img(img):

    placa_img = detectar_placa(img)
    if placa_img is not None:
        # Reconocer el texto de la placa
        texto_placa = reconocer_placa(placa_img)
        if texto_placa:
            return texto_placa
        else:
            print("No se pudo reconocer el texto de la placa")
    else:
        print("No se detectó ninguna placa en la imagen")

## 1.4. Generar fecha ficticia para entrada y salida del vehículo

In [7]:
def generar_fecha_hora():
    ahora = datetime.now()
    delta = timedelta(days=random.randint(-30, 0), hours=random.randint(0, 23), minutes=random.randint(0, 59))
    return ahora + delta

## 1.5. Generar datos aleatorios para cada matrícula

In [8]:
import random

def genear_datos_random(matricula):

    marcas_modelos = {
        "Toyota": ["Supra", "Celica", "Yaris"],
        "Ford": ["GT", "Focus", "Mustang"],
        "Honda": ["NSX", "Civic", "CR-V"],
        "Nissan": ["GTR", "350z", "240sx"],
        "Audi": ["RS7", "R8", "S5"],
        "BMW": ["330e", "M4", "i8"],
        "Mercedes": ["C63S", "GT", "A45"],
        "Tesla": ["3", "Y", "X"],
        "Porsche": ["911", "Cayman", "Taycan"],

    }
    colores = ["Rojo", "Azul", "Negro", "Blanco", "Gris", "Amarillo", "Verde"]
    tipos_vehiculo = [1, 2, 3]

    fecha_entrada = generar_fecha_hora()
    fecha_salida = fecha_entrada + timedelta(seconds=random.randint(0, 100000))

    # Generar marca y modelo aleatorio
    marca = random.choice(list(marcas_modelos.keys()))
    modelo = random.choice(marcas_modelos[marca])

    # Seleccionar color y tipo de vehículo aleatorio
    color = random.choice(colores)
    tipo_vehiculo_id = random.choice(tipos_vehiculo)

    # Devolver datos generados
    car_info = {
        "matricula_id": matricula,
        "marca_ds": marca,
        "modelo_ds": modelo,
        "color_ds": color,
        "tipo_vehiculo_id": tipo_vehiculo_id,
        "camara_in_id": fecha_entrada.strftime("%Y-%m-%d %H:%M:%S"),
        "camara_out_id": fecha_salida.strftime("%Y-%m-%d %H:%M:%S")
    }
    return car_info

## 1.6. Insertar los datos de la matrícula en la base de datos

In [9]:
def insertar_matricula(car_info):

    # Inicializar cliente BigQuery
    client = bigquery.Client()

    # Definir la tabla
    table_id = "aiparking-451016.00_aiparking.vehiculos"

    # Crear el diccionario con los datos
    fila = {
        "matricula_id": car_info["matricula_id"],
        "marca_ds": car_info["marca_ds"],
        "modelo_ds": car_info["modelo_ds"],
        "color_ds": car_info["color_ds"],
        "tipo_vehiculo_id": car_info["tipo_vehiculo_id"],
        "camara_in_id": car_info["camara_in_id"],
        "camara_out_id": car_info["camara_out_id"]
    }

    # Insertar la fila
    errores = client.insert_rows_json(table_id, [fila])

    if errores == []:
        print("Matrícula insertada correctamente.")
    else:
        print("Errores al insertar:", errores)

## PIPELINE

In [10]:
def main_OCR(img=None):

    # 1. Insertar imagen
    if img is None:
      img = insert_img()

    # 2. Detectar texto placa
    matricula = detectar_placa_img(img)

    # 3. Generar datos aleatorios
    car_info = genear_datos_random(matricula)

    # 4. Insertar datos en BigQuery
    insertar_matricula(car_info)

    return print(car_info)

In [13]:
main_OCR()

Por favor, selecciona una imagen del vehículo:


Saving E1,CAM1,250210080944560,GC4532CB,9996.jpg to E1,CAM1,250210080944560,GC4532CB,9996 (12).jpg
Errores al insertar: [{'index': 0, 'errors': [{'reason': 'invalid', 'location': 'camara_in_id', 'debugInfo': '', 'message': 'no such field: camara_in_id.'}]}]
{'matricula_id': 'GC4532CB', 'marca_ds': 'Honda', 'modelo_ds': 'CR-V', 'color_ds': 'Azul', 'tipo_vehiculo_id': 1, 'camara_in_id': '2025-02-21 01:29:19', 'camara_out_id': '2025-02-21 11:41:57'}


## 1. MODELO LLM

In [14]:
from openai import OpenAI
import json
import getpass

api_key = getpass.getpass("Enter your OpenAI API Key:")

client_openai = OpenAI(api_key = api_key)

Enter your OpenAI API Key:··········


### 1.1. Input del usuario

In [15]:
def user_input():
    question = input("Escriba su consulta:")
    return question

### 1.2. Prompt Engineering con ChatGPT

In [16]:
def input_format(question, model="gpt-3.5-turbo", temperature=0):

    prompt_template = """
Eres un experto de SQL para una base de datos de registro de vehiculos. Tu tarea es convertir una pregunta de lenguaje natural a una consulta SQL valida.

## **ESTRUCTURA DE LA BASE DE DATOS**
Nombre del proyecto: 'aiparking-451016'
Nombre de la tabla: '00_aiparking.vehiculos'

- matricula_id STRING
- parking_id STRING
- fecha_registro_dt DATETIME
- camara_in_id STRING
- camara_out_id STRING
- tipo_vehiculo_id STRING
- marca_ds STRING
- modelo_ds STRING
- color_ds STRING
- tarifa_id STRING

## **TAREA**
Responde UNICAMENTE con la consulta en SQL.

## **EJEMPLOS**
Pregunta: "Cual es el modelo del vehiculo con la matricula 5489 HEB?"
Respuesta: SELECT modelo_ds FROM `aiparking-451016.00_aiparking.vehiculos` WHERE matricula_id = '5489 HEB';

Pregunta: "Coches de color Gris, Blanco o Azul y que sean de tipo 1"
Respuesta: SELECT * FROM `aiparking-451016.00_aiparking.vehiculos` WHERE color_ds IN ('Gris', 'Blanco', 'Azul') AND tipo_vehiculo_id = '1';

Pregunta: "A qué hora entró el coche con matrícula 1234ABC el día de ayer?"
Respuesta: SELECT fecha_registro_dt FROM `aiparking-451016.00_aiparking.vehiculos` WHERE matricula_id = '1234ABC' AND DATE(fecha_registro_dt) = DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY);;

## **NUEVA PREGUNTA DEL USUARIO:**
"{question}"

## **RESPUESTA EN SQL:**
"""
    prompt = prompt_template.format(question=question)
    messages = [{"role": "user", "content": prompt}]
    response = client_openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    sql_query = response.choices[0].message.content
    return sql_query


* Ejemplo 1

In [17]:
question_1 = "qué coches con matrícula que acaba en L entraron al parking el día 25/01/2025 entre las 07:00am y las 09:30am?"

In [18]:
response_1 = input_format(question_1)
print(response_1)

SELECT * FROM `aiparking-451016.00_aiparking.vehiculos` 
WHERE matricula_id LIKE '%L' 
AND fecha_registro_dt BETWEEN '2025-01-25 07:00:00' AND '2025-01-25 09:30:00';


### 1.3. Acceso a la Database de BigQuery

In [19]:
from google.colab import auth
auth.authenticate_user()

In [20]:
from google.cloud import bigquery

# Creamos un cliente de BigQuery
client = bigquery.Client(project='aiparking-451016')

In [21]:
import pandas as pd

def sql_ans(query):

      # Consulta SQL
      query_job = client.query(query)

      results = query_job.result()

      # Convertir los resultados a un DataFrame de Pandas
      df = results.to_dataframe()

      return df

### 1.4. Pipeline

In [23]:
def main_LLM(question=None):

    # 1. Pregunta del usuario
    if question is None:
      question = user_input()

    # 2. Conversión lenguaje natural a SQL
    query = input_format(question=question, model="gpt-3.5-turbo", temperature=0)

    # 3. Acceso al Database de BigQuery
    answer = sql_ans(query)

    # 6. Mostrar la salida
    print(answer)

    return answer

In [25]:
main_LLM()

Escriba su consulta:dime los coches de color azul, blanco o gris de tipo 1
  matricula_id marca_ds modelo_ds color_ds tipo_vehiculo_id
0      1234MNO                      Blanco                1
1      0225KKL    Honda     Civic     Azul                1


,matricula_id,marca_ds,modelo_ds,color_ds,tipo_vehiculo_id
0,1234MNO,,,Blanco,1
1,0225KKL,Honda,Civic,Azul,1
